In [ ]:
!pip -q install datasets
!pip -q install transformers
!pip -q install transformers[sentencepiece]

In [ ]:
# load dataset & train-test split

from datasets import load_dataset

train_ds, test_ds = load_dataset('squad', split=['train[:500]', 'validation[:200]'])

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_ds.features

{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [ ]:
import pandas as pd

df = pd.DataFrame(train_ds, columns=train_ds.features)
df

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
...,...,...,...,...,...
495,56d4c1c12ccc5a1400d831e6,Beyoncé,Beyoncé announced a hiatus from her music care...,What did Beyoncé announce in January 2010?,"{'text': ['a hiatus'], 'answer_start': [18]}"
496,56d4c1c12ccc5a1400d831e7,Beyoncé,Beyoncé announced a hiatus from her music care...,Who suggested the hiatus for Beyoncé?,"{'text': ['her mother'], 'answer_start': [74]}"
497,56d4c1c12ccc5a1400d831e8,Beyoncé,Beyoncé announced a hiatus from her music care...,Who did Beyoncé part ways with during her hiatus?,"{'text': ['her father'], 'answer_start': [168]}"
498,56d4c1c12ccc5a1400d831e9,Beyoncé,Beyoncé announced a hiatus from her music care...,How long did her hiatus last?,"{'text': ['nine months'], 'answer_start': [244]}"


In [ ]:
train_ds['context'][0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [ ]:
train_ds['question'][0]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [ ]:
train_ds['answers'][0]

{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}

In [ ]:
# train-val split

splits = train_ds.train_test_split(test_size=0.1)

train_ds = splits['train']
val_ds = splits['test']

In [ ]:
train_ds.dataset_size

89819400

In [ ]:
# # data preprocessing

# train_ds = train_ds.map(
#     lambda x: {
#         'id': x['id'],
#         'context': x['context'],
#         'answers': {
#             **x['answers'],
#             **{'answer_end': [x['answers']['answer_start'][0] + len(x['answers']['text'][0])]}
#         },
#         'question': x['question'],
#         'title': x['title']
#     }
# )

In [ ]:
# tokenizer

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
train_ds = train_ds.map(
    lambda x: tokenizer(
            x['question'], x['context'],
            max_length=512, padding='max_length',
            truncation=True
        ), batched=True, batch_size=32
)

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
train_ds.features

{'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
df = pd.DataFrame(train_ds, columns=train_ds.features)
df

,answers,attention_mask,context,id,input_ids,question,title,token_type_ids
0,"{'text': ['11 million'], 'answer_start': [419]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Beyoncé's first solo recording was a feature o...,56be90ee3aeaaa14008c90e6,"[101, 2129, 2116, 2038, 1000, 20754, 1999, 229...","How many has ""Dangerously in Love"" sould world...",Beyoncé,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"{'text': ['in the late 1990s'], 'answer_start'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,"[101, 2043, 2106, 20773, 2707, 3352, 2759, 102...",When did Beyonce start becoming popular?,Beyoncé,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ..."
2,"{'text': ['1851–1921'], 'answer_start': [353]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","Father Joseph Carrier, C.S.C. was Director of ...",5733b0fb4776f41900661043,"[101, 2054, 2001, 1996, 26462, 1997, 2198, 140...",What was the lifespan of John Augustine Zahm?,University_of_Notre_Dame,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ..."
3,"{'text': ['1987–2005'], 'answer_start': [64]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",In the 18 years under the presidency of Edward...,573394c84776f41900660ddd,"[101, 2076, 2054, 2086, 2001, 3487, 6670, 6977...",During what years was Edward Malloy president ...,University_of_Notre_Dame,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,"{'text': ['the 1940s'], 'answer_start': [963]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",The Lobund Institute grew out of pioneering re...,5733b2fe4776f41900661091,"[101, 2105, 2054, 2051, 2106, 8840, 27265, 199...",Around what time did Lobund of Notre Dame beco...,University_of_Notre_Dame,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...,...,...,...,...
445,"{'text': ['2003'], 'answer_start': [526]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56d43c5f2ccc5a1400d830ac,"[101, 2043, 2106, 20773, 2713, 20754, 1999, 22...",When did Beyoncé release Dangerously in Love?,Beyoncé,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
446,"{'text': ['Horizon League'], 'answer_start': [...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Notre Dame teams are known as the Fighting Iri...,5733c1a94776f419006611a7,"[101, 2029, 2223, 2106, 10289, 8214, 3554, 349...",Which league did Notre Dame Fighting Irish tea...,University_of_Notre_Dame,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
447,"{'text': ['Big East Conference'], 'answer_star...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Notre Dame teams are known as the Fighting Iri...,5733c1a94776f419006611aa,"[101, 1999, 2054, 3034, 2106, 1996, 3554, 3493...",In what conference did the Fighting Irish take...,University_of_Notre_Dame,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
448,"{'text': ['Mariah Carey'], 'answer_start': [517]}","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","At the 52nd Annual Grammy Awards, Beyoncé rece...",56bf9b57a10cfb14005511b4,"[101, 2040, 2106, 2027, 5495, 2007, 2005, 2416...",Who did they tie with for six top songs?,Beyoncé,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ..."


In [ ]:
# rename column
dataset['train'] = dataset['train'].rename_column('title', 'topic')

# filter
dataset['train'] = dataset['train'].filter(
    lambda x: x['topic'] != 'University_of_Notre_Dame'
)

# delete feature
dataset['train'] = dataset['train'].remove_columns([
    'answers', 'context', 'id', 'question', 'topic'
])


In [ ]:
# load metric

from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets
# tokenized_datasets.remove_columns(['idx','sentence1','sentence2'])
# tokenized_datasets.rename_column('label','labels')
# tokenized_datasets.with_format('tensorflow')

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 50000
    })
})

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))
# full_train_dataset = tokenized_datasets["train"]
# full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=39, training_loss=0.5010326092059796, metrics={'train_runtime': 1576.3376, 'train_samples_per_second': 0.19, 'train_steps_per_second': 0.025, 'total_flos': 78933316608000.0, 'train_loss': 0.5010326092059796, 'epoch': 3.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


{'eval_accuracy': 0.79,
 'eval_loss': 0.5128568410873413,
 'eval_runtime': 170.7289,
 'eval_samples_per_second': 0.586,
 'eval_steps_per_second': 0.076}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer